<a href="https://colab.research.google.com/github/mahmoudmagdyhassan/Mahmoud-/blob/main/car%20prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Read the data
df = pd.read_csv('/content/train-data.csv')
df.head(2)


In [8]:
df.drop('Unnamed: 0' , axis =1 , inplace = True )
df.drop('Name', axis=1, inplace=True)
df.drop("New_Price", axis=1,inplace=True)

In [9]:
df['Mileage'] = df['Mileage'].str.rstrip(' km/kg')
df['Mileage'] = df['Mileage'].str.rstrip(' kmpl')
df['Engine'] = df['Engine'].str.rstrip(' CC')
df['Power'] = df['Power'].str.rstrip(' bhp')


In [10]:

df['Mileage'] = df['Mileage'].astype('float')
df['Engine'] = df['Engine'].astype('float')
df['Power']= df['Engine'].astype('float')

In [ ]:
round((df.isna().mean()*100),2)

In [12]:
df['Mileage'].fillna(df['Mileage'].mean(), inplace=True)
df['Engine'].fillna(df['Engine'].mean(), inplace=True)
df['Power'].fillna(df['Power'].mean(), inplace=True)
df['Seats'].fillna(df['Seats'].mean(), inplace=True)
round((df.isna().mean()*100),2)

Location             0.0
Year                 0.0
Kilometers_Driven    0.0
Fuel_Type            0.0
Transmission         0.0
Owner_Type           0.0
Mileage              0.0
Engine               0.0
Power                0.0
Seats                0.0
Price                0.0
dtype: float64

In [13]:
import plotly.express as px
def colpie (col ):
    fig = px.pie(data_frame = df , names = col)
    fig.show()

In [ ]:
colpie('Location')


In [ ]:
colpie('Fuel_Type')

In [ ]:
colpie('Transmission')

In [ ]:
colpie('Owner_Type')

In [ ]:
num_cols = df.select_dtypes(exclude = 'O').columns.tolist()
num_cols
print("num_cols:",num_cols)
for col in num_cols:
    print(df[col].value_counts())
    print('------------------------------------------------------------')


In [ ]:


plt.figure(figsize = (20 , 40))
for i , col in enumerate (num_cols  ,1) :
    plt.subplot((len(num_cols)//2+1) , 3 , i )
    sns.histplot(data = df , x = col)

In [ ]:
fuel_price = df.groupby('Fuel_Type')['Price'].mean().sort_values(ascending=False)
fig = px.bar(fuel_price , color = fuel_price.index )
fig.show()

In [ ]:
trans_price = df.groupby('Transmission')['Price'].mean().sort_values(ascending=False)
fig = px.bar(trans_price , color = trans_price.index )
fig.show()

In [ ]:
owner_price = df.groupby('Owner_Type')['Price'].mean().sort_values(ascending=False)
fig = px.bar(owner_price , color = owner_price.index )
fig.show()

In [ ]:
year_price = df.groupby('Year')['Price'].mean()
fig = px.bar(year_price , color = year_price.index )
fig.show()

In [ ]:
sns.pairplot(df );

In [ ]:
sns.boxplot( x='Kilometers_Driven', data=df);


In [ ]:
sns.boxplot( x='Price', data=df);


In [ ]:
sns.boxplot( x='Power', data=df);

In [ ]:
sns.boxplot( x='Engine', data=df);


In [ ]:
df.drop(df[df['Kilometers_Driven'] > 100000].index, axis = 0, inplace = True)
df.drop(df[df['Power'] > 480].index, axis = 0, inplace = True)
df.drop(df[df['Engine'] > 5000].index, axis = 0, inplace = True)

In [15]:
df = pd.get_dummies(df, columns = ['Location', 'Fuel_Type','Transmission','Owner_Type'])
df.head(2)

,Year,Kilometers_Driven,Mileage,Engine,Power,Seats,Price,Location_Ahmedabad,Location_Bangalore,Location_Chennai,...,Fuel_Type_Diesel,Fuel_Type_Electric,Fuel_Type_LPG,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual,Owner_Type_First,Owner_Type_Fourth & Above,Owner_Type_Second,Owner_Type_Third
0,2010,72000,26.60,998.0,998.0,5.0,1.75,0,0,0,...,0,0,0,0,0,1,1,0,0,0
1,2015,41000,19.67,1582.0,1582.0,5.0,12.50,0,0,0,...,1,0,0,0,0,1,1,0,0,0


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(df)
df = pd.DataFrame(scaled_data,
						columns=df.columns)
df.head()

In [ ]:
x = df.drop('Price', axis=1).values

y = df['Price'].values

print(x.shape)
print(y.shape)


In [27]:

class LinearRegression:

    def __init__(self, l_rate=0.0001, iterations=1000):
        self.l_rate = l_rate
        self.iterations = iterations

    def fitGD(self, x, y):
        self.cost = []
        self.theta = np.zeros((1 + x.shape[1]))
        n = x.shape[0]

        for i in range(self.iterations):
            y_pred = self.theta[0] + np.dot(x, self.theta[1:])
            mse = (1/n) * np.sum((y_pred - y)**2)
            self.cost.append(mse)


            d_theta1 = (2/n) * np.dot(x.T, (y_pred - y))
            d_theta0 = (2/n) * np.sum(y_pred - y)


            self.theta[1:] = self.theta[1:] - self.l_rate * d_theta1
            self.theta[0] = self.theta[0] - self.l_rate * d_theta0
        return self


    def predictGD(self, x):
        return self.theta[0] + np.dot(x, self.theta[1:])


    def fitNQ(self,x, y):
        z = np.ones((x.shape[0],1))
        x = np.append(z, x, axis=1)
        self.thetas = np.linalg.pinv(x.T.dot(x)).dot(x.T).dot(y)

        return self

    def predictNQ(self, x):
        z = np.ones((x.shape[0],1))
        x = np.append(z, x, axis=1)
        return np.dot(x, self.thetas)

In [28]:
def scale(x):
    x_scaled = x - np.mean(x, axis=0)
    x_scaled = x_scaled / np.std(x_scaled, axis=0)
    return x_scaled

In [29]:
x_sd= scale(x)
lr = LinearRegression()
lr.fitGD(x, y)
lr.fitNQ(x, y)

In [30]:
print("theta_0= ", lr.theta[0])
print("theta_1= ", lr.theta[1])
print("theta_2= ", lr.theta[2])

theta_0=  -2.9439300123584334e-18
theta_1=  0.052681528836546684
theta_2=  -0.0032754257716349207


In [31]:
print("theta_0= ", lr.thetas[0])
print("theta_1= ", lr.thetas[1])
print("theta_2= ", lr.thetas[2])

theta_0=  2.8519938147231194e-16
theta_1=  0.3245815950556436
theta_2=  0.000492938752243775


In [32]:
y_pred = lr.predictGD(x_sd)
y_predNQ = lr.predictNQ(x)
print("y_pred:",y_pred)
print("y_predNQ:",y_predNQ)
print("y_actual:",y)

y_pred: [-0.49200093 -0.05354941 -0.37880131 ...  0.15495565 -0.44495697
 -0.32883754]
y_predNQ: [-1.06420354  0.06352261 -0.76821486 ... -0.03869236 -0.89215109
 -0.91466472]
y_actual: [-0.69093384  0.27000402 -0.44511253 ... -0.58813584 -0.61048323
 -0.62389166]


In [33]:
errors = np.sum((y_pred - y)**2)
sst = np.sum((y - np.mean(y))**2)
r2_GD = 1 - (errors/sst)

adusted_r2_GD= 1-((1-r2_GD) * (x.shape[0] -1)/(x.shape[0] - x.shape[1] - 1))
print ("adusted_r2_GD:",adusted_r2_GD)


errors = np.sum((y_predNQ - y)**2)
sst = np.sum((y - np.mean(y))**2)
r2_NQ = 1 - (errors/sst)

adusted_r2_NQ= 1-((1-r2_NQ) * (x.shape[0] -1)/(x.shape[0] - x.shape[1] - 1))
print ("adusted_r2_NQ:",adusted_r2_NQ)

adusted_r2_GD: 0.43250263366525743
adusted_r2_NQ: 0.6466410828373206


In [34]:


from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Model Building
from sklearn.linear_model import LinearRegression

# Linear Regression
lr = LinearRegression()
lr.fit(x_train, y_train)

# Model Evaluation
from sklearn.metrics import mean_squared_error, r2_score

# Predictions
y_pred = lr.predict(x_test)

# Model Evaluation
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R2:', r2_score(y_test, y_pred))


RMSE: 0.8817338300907931
R2: 0.20934647625235359


In [35]:
from sklearn.linear_model import Ridge

ridge = Ridge()
ridge.fit(x_train, y_train)

# Predictions
y_pred = ridge.predict(x_test)

# Model Evaluation
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R2:', r2_score(y_test, y_pred))

RMSE: 0.8904101463570997
R2: 0.19370975911026278


In [36]:
# Ridge Regression
from sklearn.linear_model import Lasso

lasso = Lasso()
lasso.fit(x_train, y_train)

# Predictions
y_pred = lasso.predict(x_test)

# Model Evaluation
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R2:', r2_score(y_test, y_pred))

RMSE: 0.9917008080462937
R2: -0.0001670366620361019


In [37]:
for alpha in [0.001, 0.01, 0.1, 1, 10, 100]:
    ridge = Ridge(alpha=alpha)
    ridge.fit(x_train, y_train)
    y_pred = ridge.predict(x_test)
    print('alpha:', alpha)
    print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
    print('R2:', r2_score(y_test, y_pred))
    print('-----------------------------')

alpha: 0.001
RMSE: 0.8911017502018328
R2: 0.19245674105611532
-----------------------------
alpha: 0.01
RMSE: 0.8910955058410112
R2: 0.19246805867235595
-----------------------------
alpha: 0.1
RMSE: 0.8910330759541968
R2: 0.19258120560524072
-----------------------------
alpha: 1
RMSE: 0.8904101463570997
R2: 0.19370975911026278
-----------------------------
alpha: 10
RMSE: 0.8843149659284085
R2: 0.2047106754894269
-----------------------------
alpha: 100
RMSE: 0.8344424387918014
R2: 0.29188471286461704
-----------------------------


In [38]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

poly = make_pipeline(PolynomialFeatures(2), Ridge())
poly.fit(x_train, y_train)

# Predictions
y_pred = poly.predict(x_test)

# Model Evaluation
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R2:', r2_score(y_test, y_pred))
print('R2:', poly.score(x_test, y_test))


RMSE: 3.4552654185018543
R2: -11.141547131662353
R2: -11.141547131662353


In [ ]:
for degree in [2, 3, 4, 5]:
    poly = make_pipeline(PolynomialFeatures(degree), Ridge())
    poly.fit(x_train, y_train)
    y_pred = poly.predict(x_test)
    print('degree:', degree)
    print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))
    print('R2:', r2_score(y_test, y_pred))
    print('-----------------------------')

degree: 2
RMSE: 3.4552654185018543
R2: -11.141547131662353
-----------------------------
degree: 3
RMSE: 5.776369587554172
R2: -32.93291337169733
-----------------------------
degree: 4
RMSE: 6568.790836857631
R2: -43881560.11110972
-----------------------------
